In [ ]:
# https://www.kaggle.com/datasets/cosmos98/twitter-and-reddit-sentimental-analysis-dataset?resource=download

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import emoji

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
df = pd.read_csv('../datasets/emoji_sentiment_dataset.csv')

print(df)
print(f"El dataset tiene {df.shape[0]} filas")
print(f"El dataset tiene {df.isnull().any(axis=1).sum()} filas con valores nulos")
df_clean = df.dropna()
print(f"El dataset tiene {df_clean.isnull().any(axis=1).sum()} filas con valores nulos")

In [ ]:
df_clean.groupby('category').count().plot(kind='bar')

In [ ]:

def plot_word_cloud(terms, category):
    # Definir un título basado en el valor de la categoría
    if category == 1.0:
        title = "Positivo"
    elif category == 0.0:
        title = "Neutral"
    else:
        title = "Negativo"
    
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(terms)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'Nube de Palabras - Sentimiento {title}', fontsize=20)  # Añadir título aquí
    plt.axis("off")
    plt.show()

# Asegúrate de que los datos están limpios y los textos son strings
df_clean['clean_text'] = df_clean['clean_text'].astype(str)

# Generar nubes de palabras para cada categoría
for category in [-1.0, 0.0, 1.0]:
    subset = df_clean[df_clean['category'] == category]
    texts = " ".join(subset['clean_text'])
    tfidf_vector = TfidfVectorizer(stop_words='english', max_features=1000)
    X_sub = tfidf_vector.fit_transform([texts])
    max_words = {word: X_sub[0, idx] for word, idx in tfidf_vector.vocabulary_.items()}
    plot_word_cloud(max_words, category)  # Pasar también la categoría como argumento


In [ ]:
# Tokenización
tokenizer = Tokenizer(num_words=5000)  # Solo considera las 5000 palabras más frecuentes
tokenizer.fit_on_texts(df_clean['clean_text'])
sequences = tokenizer.texts_to_sequences(df_clean['clean_text'])

In [ ]:
# Padding
max_sequence_len = max([len(x) for x in sequences])
X = pad_sequences(sequences, maxlen=max_sequence_len)
y = to_categorical(np.asarray(df_clean['category'] + 1))  # Convertir -1, 0, 1 a 0, 1, 2 para categorías

In [ ]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo
model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=max_sequence_len),
    LSTM(64, return_sequences=True),
    Dropout(0.5),
    LSTM(64),
    Dense(3, activation='softmax')  # 3 categorías de salida
])

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, batch_size=32, epochs=3, validation_split=0.1)

# Evaluar el modelo
print(model.evaluate(X_train, y_train))


In [ ]:
# Función para obtener el nombre Unicode de un emoji
def emoji_to_unicode_name(em):
    return emoji.demojize(em)

# Lista de emojis
new_text = '😍😍🤘🤘❤️❤️👄😎😎'

# Convertir la lista a un pandas Series
new_text_series = pd.Series(new_text)

# Aplicar la función a la Series
new_text_unicode_names = new_text_series.apply(emoji_to_unicode_name).str.replace(':',' ')


# Convertir de nuevo a lista si es necesario
new_text = new_text_unicode_names.tolist()

print(new_text)

seq = tokenizer.texts_to_sequences(new_text)
padded = pad_sequences(seq, maxlen=max_sequence_len)

# Realizar la predicción
pred = model.predict(padded)
print(f'Predicción: {pred[0]}')  # Muestra la probabilidad de cada categoría

# Determinar la categoría más probable
predicted_category_index = np.argmax(pred[0])
categories = {-1: 'Negativo', 0: 'Neutral', 1: 'Positivo'}  # Categorías ajustadas para corresponder a tus etiquetas
predicted_category = categories[predicted_category_index - 1]  # ajustar el índice para -1, 0, 1
print(f'La categoría más probable es: {predicted_category} con una probabilidad de {pred[0][predicted_category_index]}')
